In [66]:
import pandas as pd
import ast

In [99]:
df=pd.read_csv(r'C:\Users\Yasaman\Arab_spring_scholarly_attention\Validating\final_annotated.csv')


In [100]:
df.iloc[208]

ID                                           https://openalex.org/W826975064
Year                                                                    2015
Field                                           Earth and Planetary Sciences
SampleGroup                                                         field_20
Title                      Automatic LiDAR-based lighting inventory in bu...
Abstract                   Construction industry is a large contributor i...
annot_by_hongyu                                                          NaN
annot_by_yasaman                                                         NaN
annotator_1                                                               []
annotator_2                                                               []
intersection_annotation                                                   []
union_annotation                                                          []
annotator_agreement                                                     True

In [101]:
df=pd.read_csv(r'C:\Users\Yasaman\Arab_spring_scholarly_attention\Validating\final_annotated.csv')
df2=pd.read_excel(r"C:\Users\Yasaman\Downloads\samples_1127.xlsx")
Map={row['ID']:row['Mentions'] for i,row in df2.iterrows()}
df.loc[(df['SampleGroup'] == 'with_mention_arab') & (df['Mentions'].isna()), 'Mentions'] = df.loc[(df['SampleGroup'] == 'with_mention_arab') & (df['Mentions'].isna()), 'ID'].map(Map)


In [102]:
df['union_annotation'] = df['union_annotation'].apply(ast.literal_eval)
df['intersection_annotation'] = df['intersection_annotation'].apply(ast.literal_eval)
df['Mentions'] = df['Mentions'].apply(ast.literal_eval)


In [103]:
df['annotator_agreement'].value_counts()

annotator_agreement
True     891
False    109
Name: count, dtype: int64

In [107]:
sample_accuracies = df.groupby("SampleGroup").apply(
    lambda x: pd.Series({
        "accuracy_union": (x.apply(lambda row: set(row['Mentions']) == set(row['union_annotation']), axis=1).mean()),
        "accuracy_intersection": (x.apply(lambda row: set(row['Mentions']) == set(row['intersection_annotation']), axis=1).mean())
    })
).reset_index()

sample_accuracies = sample_accuracies.sort_values(by='accuracy_union').reset_index(drop=True)
sample_accuracies

C:\Users\Yasaman\AppData\Local\Temp\ipykernel_21972\3424697931.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sample_accuracies = df.groupby("SampleGroup").apply(


,SampleGroup,accuracy_union,accuracy_intersection
0,with_mention_arab,0.840000,0.750000
1,with_mention,0.928571,0.775000
2,field_20,0.975000,0.959615


In [105]:
df['Mentions']=df["Mentions"].apply(lambda x: sorted(x))
df['union_annotation']=df["union_annotation"].apply(lambda x: sorted(x))
df['intersection_annotation']=df["intersection_annotation"].apply(lambda x: sorted(x))

In [106]:
sample_accuracies = df.groupby("SampleGroup").apply(
    lambda x: pd.Series({
        "jaccard_union": (
            x.apply(
                lambda row: len(set(row['Mentions']).intersection(set(row['union_annotation']))) /
                            len(set(row['Mentions']).union(set(row['union_annotation'])))
                if len(set(row['Mentions']).union(set(row['union_annotation']))) > 0 else 1,
                axis=1
            ).mean()
        ),
        "jaccard_intersection": (
            x.apply(
                lambda row: len(set(row['Mentions']).intersection(set(row['intersection_annotation']))) /
                            len(set(row['Mentions']).union(set(row['intersection_annotation'])))
                if len(set(row['Mentions']).union(set(row['intersection_annotation']))) > 0 else 1,
                axis=1
            ).mean()
        )
    })
).reset_index()

sample_accuracies = sample_accuracies.sort_values(by='jaccard_union').reset_index(drop=True)
sample_accuracies

C:\Users\Yasaman\AppData\Local\Temp\ipykernel_21972\292121645.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sample_accuracies = df.groupby("SampleGroup").apply(


,SampleGroup,jaccard_union,jaccard_intersection
0,with_mention_arab,0.905411,0.820729
1,with_mention,0.946131,0.804048
2,field_20,0.976923,0.960577


,ID,Year,Field,SampleGroup,Title,Abstract,annot_by_hongyu,annot_by_yasaman,annotator_1,annotator_2,intersection_annotation,union_annotation,annotator_agreement,Mentions,annot_by_shadi,annot_by_ozgur,annot_by_mary,annot_by_juven
208,https://openalex.org/W826975064,2015,Earth and Planetary Sciences,field_20,Automatic LiDAR-based lighting inventory in bu...,Construction industry is a large contributor i...,NaN,NaN,['-'],-,[],[-],True,[],NaN,NaN,['-'],NaN


In [82]:
df

,ID,Year,Field,SampleGroup,Title,Abstract,annot_by_hongyu,annot_by_yasaman,annotator_1,annotator_2,intersection_annotation,union_annotation,annotator_agreement,Mentions,annot_by_shadi,annot_by_ozgur,annot_by_mary,annot_by_juven
0,https://openalex.org/W1588375827,2015,Immunology and Microbiology,with_mention_arab,Detection of Schistosoma mansoni and Schistoso...,The present study describes a real-time PCR ap...,['yem'],['yem'],['yem'],['yem'],[yem],[yem],True,[yem],NaN,NaN,NaN,NaN
1,https://openalex.org/W16358656,2010,"Business, Management and Accounting",with_mention_arab,Accounting information systems in a centrally ...,The paper has a twofold purpose. First it high...,['lby'],['lby'],['lby'],['lby'],[lby],[lby],True,[lby],NaN,NaN,NaN,NaN
2,https://openalex.org/W1963566746,2014,"Biochemistry, Genetics and Molecular Biology",with_mention_arab,Distinctive genetic signatures of <i>Alu</i>/S...,The Middle East and Mediterranean represents o...,"['bhr', 'irn', 'lby']","['bhr', 'irn', 'lby']","['bhr', 'irn', 'lby']","['bhr', 'irn', 'lby']","[bhr, irn, lby]","[bhr, irn, lby]",True,"[bhr, irn, lby]",NaN,NaN,NaN,NaN
3,https://openalex.org/W1966506681,2014,"Biochemistry, Genetics and Molecular Biology",with_mention_arab,EnterohaemorrhagicEscherichia coliO157: a surv...,Zoonotic Escherichia coli O157 pathogen repres...,['lby'],['lby'],['lby'],['lby'],[lby],[lby],True,[lby],NaN,NaN,NaN,NaN
4,https://openalex.org/W1968999285,2014,Earth and Planetary Sciences,with_mention_arab,A Space View of Radar Archaeological Marks: Fi...,With the development of Synthetic Aperture Rad...,"['ita', 'lby']","['ita', 'lby']","['ita', 'lby']","['ita', 'lby']","[ita, lby]","[ita, lby]",True,"[ita, lby]",NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,https://openalex.org/W2050533565,2013,Health Professions,with_mention_arab,Punitive military action in syria will just pr...,As news of the recent alleged chemical weapons...,NaN,NaN,['syr'],['syr'],[syr],[syr],True,[syr],['syr'],NaN,['syr'],NaN
996,https://openalex.org/W2609606711,2017,Computer Science,with_mention_arab,Human Action Prediction Using Sentiment Analys...,There is a rapid increase of mass demonstratio...,"['usa', 'egy', 'zaf']",NaN,"['usa', 'egy', 'zaf']","['usa', 'egy', 'zaf']","[egy, usa, zaf]","[egy, usa, zaf]",True,"[egy, usa, zaf]",NaN,NaN,NaN,"['usa', 'egy', 'zaf']"
997,https://openalex.org/W2892804921,2018,Engineering,with_mention_arab,Global positioning systems meteorology over Mo...,Global positioning systems (GPSs) have become ...,['mar'],NaN,['mar'],['mar'],[mar],[mar],True,[mar],NaN,NaN,NaN,['mar']
998,https://openalex.org/W2051745229,2012,Earth and Planetary Sciences,with_mention_arab,Surface Geostrophic Circulation of the Mediter...,Abstract Drifter observations and satellite-de...,NaN,NaN,"['isr', 'lbn', 'syr', 'tur']","['isr', 'lbn', 'syr', 'tur']","[isr, lbn, syr, tur]","[isr, lbn, syr, tur]",True,"[gib, isr, lbn, syr, tur]",NaN,NaN,"['isr', 'lbn', 'syr', 'tur']","['isr', 'lbn', 'syr', 'tur']"
